In [0]:
"""Notebook MVP 04 - Gerar painel consolidado multi-KM (racional de ciclos).

Este notebook:
- Le as tabelas Gold geradas no MVP 03:
  - gold_conformidade_ciclo_km.parquet
  - gold_conformidade_periodo_km.parquet
- Seleciona um conjunto de KMs alvo para o painel.
- Monta uma planilha Excel no formato proximo ao exemplo:
  - linhas = ciclos (1o periodo: 1-4, 2o periodo: 5-8)
  - colunas = KMs alvo + colunas de resumo (Media, Atendimento, Saldo)

Obs.: Focado em um mes (Mes = AAAA-MM). Para o MVP usaremos o mes presente nos dados.
"""

import os
from pathlib import Path

import numpy as np
import pandas as pd

WORKSPACE_BASE = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp"
GOLD_DIR = f"{WORKSPACE_BASE}/gold"
BRONZE_DIR = f"{WORKSPACE_BASE}/dados_bronze"

ARQ_GOLD_CICLO = f"{GOLD_DIR}/gold_conformidade_ciclo_km.parquet"
ARQ_GOLD_PERIODO = f"{GOLD_DIR}/gold_conformidade_periodo_km.parquet"

# Parametros para escolha dos KMs (mesma ideia do listar_top_kms.py)
RODOVIA_TOP = "BR-153/GO"
SENTIDO_TOP = "S"
QTD_TOP_KM = 11


def converter_km(km_str: object) -> float:
    """Converte formato XXX+YYY para float (XXX.YYY)."""
    s = str(km_str)
    if s.strip() == "" or s.lower() == "nan":
        return np.nan
    if "+" in s:
        partes = s.split("+")
        if len(partes) == 2:
            try:
                return float(partes[0]) + float(partes[1]) / 1000.0
            except Exception:
                return np.nan
    try:
        return float(s.replace("+", ""))
    except Exception:
        return np.nan

print("Gold ciclo:", ARQ_GOLD_CICLO)
print("Gold periodo:", ARQ_GOLD_PERIODO)

if not os.path.exists(ARQ_GOLD_CICLO) or not os.path.exists(ARQ_GOLD_PERIODO):
    raise FileNotFoundError("Arquivos Gold nao encontrados. Gere antes com o notebook 03.")

print("Lendo tabelas Gold...")
df_ciclo = pd.read_parquet(ARQ_GOLD_CICLO)
df_periodo = pd.read_parquet(ARQ_GOLD_PERIODO)

print("Linhas df_ciclo:", len(df_ciclo))
print("Linhas df_periodo:", len(df_periodo))

# Para simplicidade do MVP, vamos pegar o primeiro mes disponivel
meses_disponiveis = sorted(df_ciclo["Mes"].unique())
if not meses_disponiveis:
    raise ValueError("Nenhum Mes encontrado em df_ciclo.")

MES_ALVO = meses_disponiveis[0]
print("Mes alvo para o painel:", MES_ALVO)

# Calcular lista de KMs alvo a partir do Bronze (reproduzindo listar_top_kms.py)
# Tentar encontrar arquivo Bronze em varios caminhos possiveis
caminhos_bronze_possiveis = [
    f"{BRONZE_DIR}/ciclos_concatenados.parquet",
    f"{WORKSPACE_BASE}/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/dados_bronze/ciclos_concatenados.parquet",
]

ARQ_BRONZE = None
for caminho_teste in caminhos_bronze_possiveis:
    if os.path.exists(caminho_teste):
        ARQ_BRONZE = caminho_teste
        print(f"[OK] Arquivo Bronze encontrado em: {caminho_teste}")
        break

if ARQ_BRONZE is None:
    raise FileNotFoundError(f"Arquivo Bronze nao encontrado. Tentou: {caminhos_bronze_possiveis}")

print("\nLendo Bronze para obter top KMs (logica listar_top_kms.py)...")
df_bronze = pd.read_parquet(ARQ_BRONZE)

rod_col = "SiglaRodovia" if "SiglaRodovia" in df_bronze.columns else ("Rodovia" if "Rodovia" in df_bronze.columns else None)
if rod_col is None:
    raise ValueError("Coluna de rodovia nao encontrada no Bronze")

sent_col = "Sentido" if "Sentido" in df_bronze.columns else None
if sent_col is None:
    raise ValueError("Coluna 'Sentido' nao encontrada no Bronze")

km_conv = df_bronze["KM"].apply(converter_km)
mask = (df_bronze[rod_col] == RODOVIA_TOP) & (df_bronze[sent_col] == SENTIDO_TOP)
serie = km_conv[mask].dropna().round(2)

vc = serie.value_counts().head(QTD_TOP_KM)
lista_km = [float(x) for x in list(vc.index)]

print(f"Top {QTD_TOP_KM} KMs (BR-153/GO, S):", lista_km)

if not lista_km:
    raise ValueError("Nenhum KM encontrado pelo criterio listar_top_kms.")

# Agora filtrar dados Gold do mes alvo e dos KMs selecionados (otimizacao)
print("\nFiltrando dados do mes alvo e KMs selecionados nos Gold...")
ciclo_mes_temp = df_ciclo[df_ciclo["Mes"] == MES_ALVO].copy()
periodo_mes_temp = df_periodo[df_periodo["Mes"] == MES_ALVO].copy()

print(f"Antes do filtro de KM: {len(ciclo_mes_temp)} linhas em ciclo_mes")
print(f"Antes do filtro de KM: {len(periodo_mes_temp)} linhas em periodo_mes")

ciclo_mes = ciclo_mes_temp[ciclo_mes_temp["KM"].isin(lista_km)].copy()
periodo_mes = periodo_mes_temp[periodo_mes_temp["KM"].isin(lista_km)].copy()

print(f"Após filtro de KM: {len(ciclo_mes)} linhas em ciclo_mes")
print(f"Após filtro de KM: {len(periodo_mes)} linhas em periodo_mes")

# Parametros de limites
LIMITE_INDIVIDUAL = 180.0
LIMITE_GERAL = 60.0

# Vamos montar duas matrizes: uma para P1 (ciclos 1-4) e outra para P2 (5-8)

def montar_bloco_periodo(periodo: str) -> pd.DataFrame:
    """Monta bloco de 4 linhas (ciclos) x KMs + colunas resumo para um periodo (P1 ou P2).
    
    Versão otimizada usando pivot ao invés de loops.
    """
    if periodo == "P1":
        ciclos = [1, 2, 3, 4]
    else:
        ciclos = [5, 6, 7, 8]
    
    # Filtrar dados do período
    df_periodo_filtrado = ciclo_mes[
        (ciclo_mes["Periodo"] == periodo) & (ciclo_mes["Ciclo"].isin(ciclos))
    ].copy()
    
    # Criar pivot: linhas = Ciclo, colunas = KM, valores = Media_Posicao_no_Ciclo
    pivot = df_periodo_filtrado.pivot_table(
        index="Ciclo",
        columns="KM",
        values="Media_Posicao_no_Ciclo",
        aggfunc="first"  # deve ter apenas 1 valor por Ciclo x KM
    )
    
    # Garantir que todos os ciclos e KMs estão presentes
    pivot = pivot.reindex(ciclos)
    pivot = pivot.reindex(columns=lista_km)
    
    # Renomear colunas para formato KM_X
    pivot.columns = [f"KM_{km}" for km in pivot.columns]
    
    # Resetar índice para ter Ciclo como coluna
    df_bloco = pivot.reset_index()
    
    # Calcular média entre KMs na linha do ciclo
    colunas_km = [c for c in df_bloco.columns if c.startswith("KM_")]
    df_bloco["Media"] = df_bloco[colunas_km].mean(axis=1)
    
    # Calcular atendimento e saldo
    df_bloco["Atendimento_180"] = np.where(
        df_bloco["Media"] <= LIMITE_INDIVIDUAL, "Sim", "Nao"
    )
    df_bloco["Saldo"] = df_bloco["Media"] - LIMITE_INDIVIDUAL
    
    # Reordenar colunas: Ciclo, KM_*, Media, Atendimento_180, Saldo
    ordem_cols = ["Ciclo"] + colunas_km + ["Media", "Atendimento_180", "Saldo"]
    df_bloco = df_bloco[ordem_cols]
    
    return df_bloco

print("Montando bloco P1...")
bloco_p1 = montar_bloco_periodo("P1")
print(bloco_p1.head())

print("Montando bloco P2...")
bloco_p2 = montar_bloco_periodo("P2")
print(bloco_p2.head())

# Calcular linha de resumo "Cenario Concessionaria" por periodo

def montar_resumo_periodo(bloco: pd.DataFrame) -> dict:
    """Calcula resumo do periodo a partir do bloco de ciclos."""
    resumo = {"Cenario": "Cenario Concessionaria"}
    # media dos ciclos (coluna Media)
    medias_validas = bloco["Media"].dropna()
    media_periodo = float(medias_validas.mean()) if len(medias_validas) > 0 else np.nan
    resumo["Media"] = media_periodo
    # saldo do periodo = soma dos saldos dos 4 ciclos
    saldos_validos = bloco["Saldo"].dropna()
    saldo_periodo = float(saldos_validos.sum()) if len(saldos_validos) > 0 else np.nan
    resumo["Saldo"] = saldo_periodo
    # atendimento do periodo
    resumo["Atendimento_60"] = "Sim" if not np.isnan(saldo_periodo) and saldo_periodo <= LIMITE_GERAL else "Nao"
    return resumo

resumo_p1 = montar_resumo_periodo(bloco_p1)
resumo_p2 = montar_resumo_periodo(bloco_p2)

print("Resumo P1:", resumo_p1)
print("Resumo P2:", resumo_p2)

# Exportar para Excel em um formato proximo ao painel original

ARQ_PAINEL = f"{GOLD_DIR}/painel_consolidado_mvp_{MES_ALVO.replace('-', '')}.xlsx"
print("Arquivo de painel de saida:", ARQ_PAINEL)

# Instalar openpyxl se necessario
try:
    import openpyxl
except ImportError:
    print("Instalando openpyxl...")
    %pip install openpyxl
    import openpyxl

with pd.ExcelWriter(ARQ_PAINEL) as writer:
    # Guia P1
    # Tabela de ciclos
    bloco_p1.to_excel(writer, sheet_name="P1", index=False, startrow=1)
    # Linha de resumo P1 logo apos as 4 linhas de ciclos
    df_resumo_p1 = pd.DataFrame([resumo_p1])
    df_resumo_p1.to_excel(writer, sheet_name="P1", index=False, startrow=1 + len(bloco_p1) + 2)
    
    # Guia P2
    bloco_p2.to_excel(writer, sheet_name="P2", index=False, startrow=1)
    df_resumo_p2 = pd.DataFrame([resumo_p2])
    df_resumo_p2.to_excel(writer, sheet_name="P2", index=False, startrow=1 + len(bloco_p2) + 2)

if os.path.exists(ARQ_PAINEL):
    tamanho_mb = os.path.getsize(ARQ_PAINEL) / (1024 * 1024)
    print(f"[OK] Painel consolidado MVP gerado em: {ARQ_PAINEL} ({tamanho_mb:.2f} MB)")
else:
    print("[ERRO] Arquivo nao foi criado!")

print("\n=== CONCLUSAO ===")
print("Painel consolidado MVP gerado com sucesso!")
print(f"Arquivo: {ARQ_PAINEL}")

Gold ciclo: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_ciclo_km.parquet
Gold periodo: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/gold/gold_conformidade_periodo_km.parquet
Lendo tabelas Gold...
Linhas df_ciclo: 140960
Linhas df_periodo: 119117
Mes alvo para o painel: 2025-05
[OK] Arquivo Bronze encontrado em: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/dados_bronze/ciclos_concatenados.parquet

Lendo Bronze para obter top KMs (logica listar_top_kms.py)...
Top 11 KMs (BR-153/GO, S): [87.26, 341.99, 129.84, 295.21, 342.02, 212.76, 170.98, 422.42, 342.0, 212.77, 170.97]

Filtrando dados do mes alvo e KMs selecionados nos Gold...
Antes do filtro de KM: 140960 linhas em ciclo_mes
Antes do filtro de KM: 119117 linhas em periodo_mes
Após filtro de KM: 88 linhas em ciclo_mes
Após filtro de KM: 22 linhas em periodo_mes
Montando bloco P1...
   Ciclo    KM_87.26   KM_341.99  ...     